<a href="https://colab.research.google.com/github/bourke-floyd-pf/material-ui/blob/master/notebooks/Platform_Ads_ML_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook used to setup / backfill agg Ads data for ML projects
### Authentication & Project Configuration Notes

This notebook now supports running both in Google Colab and a local environment.

Recommended local setup:
1. Install gcloud CLI and authenticate for ADC:
   - `gcloud auth application-default login`
2. (Alternative) Use a service account key:
   - Create key JSON and set `export GOOGLE_APPLICATION_CREDENTIALS=/path/key.json`

In [3]:
import google.auth
from google.auth.exceptions import DefaultCredentialsError

# PROJECT_ID = "analytics-204820"
PROJECT_ID = "pf-data-dev"
! gcloud config set project $PROJECT_ID
%env GCLOUD_PROJECT=$PROJECT_ID

creds, detected_project = google.auth.default(quota_project_id=PROJECT_ID)
if detected_project:
    print(f"Using Application Default Credentials for project: {detected_project}")
else:
    print("ADC credentials found (no project auto-detected). Will rely on PROJECT_ID variable.")

Updated property [core/project].
env: GCLOUD_PROJECT=pf-data-dev
Using Application Default Credentials for project: pf-data-dev


In [4]:
import os
import numpy as np
import pandas as pd
import tempfile
from google.cloud import bigquery as bq
from google.api_core.exceptions import GoogleAPIError
from pathlib import Path
from typing import Optional
# import tensorflow as tfpyhton

In [5]:
DATA_DIR = os.path.join(tempfile.gettempdir(), 'ads_data') # Storage directory
DATASET_ID = 'sandbox_ads'
DATASET_LOC = 'us'
bq_client = bq.Client(project=PROJECT_ID)
bq_dataset = bq.Dataset(bq.dataset.DatasetReference(PROJECT_ID, DATASET_ID))
bq_dataset.location = DATASET_LOC

In [6]:
def bigquery_df(query_path, lookback_days: int = 14, dry_run: bool = True, use_query_cache: bool = False) -> Optional[pd.DataFrame]:

  if (not dry_run):
      # always do a dry run first when not in dry run mode
      bigquery_df(query_path, lookback_days=lookback_days, dry_run=True, use_query_cache=use_query_cache)

  with open(query_path, 'r') as f:
      sql_text = f.read()
  job_config = bq.QueryJobConfig(
      query_parameters=[
          bq.ScalarQueryParameter("lookback_days", "INT64", lookback_days)
      ],
      dry_run=dry_run,
      use_query_cache=use_query_cache
  )
  job = bq_client.query(sql_text, job_config=job_config)
  if dry_run:
      print(f"Dry run OK.\nThis SQL will process ~{job.total_bytes_processed/1e9:,.2f} GB when run.")
  else:
      result = job.result()
      print(f"Table refreshed with lookback_days={lookback_days}.\nThis SQL processed ~{job.total_bytes_processed/1e9:,.2f} GB.")
      return result.to_dataframe()

In [8]:
test_df = bigquery_df('data/example.sql', lookback_days=3, dry_run=False)

Dry run OK.
This SQL will process ~0.00 GB when run.
Table refreshed with lookback_days=3.
This SQL processed ~0.00 GB.


In [9]:
test_df.head(5)

,event_date,player_id,platform_amazon,platform_android,platform_facebook,platform_ios,platform,build_min,build,sequence_id_min,...,loading_time_max,loading_time_count,loading_time_start_avg,loading_time_start_first_session,loading_time_sum,loading_time_start_sum,iap_rev_ad_remove_temporary,iap_txns_ad_remove_temporary,ad_remove_temp_days_remaining,blacklist
0,2025-10-02,b9ea42d9-eb1f-4c71-b156-155be9973f08,0,1,0,1,ios,2022,2022,3905,...,0.073,1,4.778664,4.778664,0.073,4.778664,NaN,<NA>,-1,<NA>
1,2025-10-03,38b5be6f-f4fe-4cba-b340-8bc8c8b4c7e3,0,0,0,1,ios,2869,2869,31,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,-1,<NA>
2,2025-10-02,88fb6296-8040-40a9-bce5-18ea988e5deb,0,0,0,1,ios,2861,2861,31,...,0.067,3,2.032556,2.007514,0.183,6.097669,NaN,<NA>,-1,<NA>


In [10]:
df = bigquery_df('data/create_or_replace_table.sql', 14, dry_run=False)

Dry run OK.
This SQL will process ~18.89 GB when run.
Table refreshed with lookback_days=14.
This SQL processed ~3.70 GB.


In [11]:
df.head(5)

""
